# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
# Your code here
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [8]:
# Your code here
from sklearn.metrics import mean_squared_error

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

residuals_train = y_train - y_hat_train
residuals_test = y_test - y_hat_test

mse_train = mean_squared_error(y_train, y_hat_train)
mse_test = mean_squared_error(y_test, y_hat_test)

mse_train, mse_test


(0.1566681017343483, 0.18879538302023618)

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [20]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    # add 1 to fold size to account for leftovers??
    
    base_size = len(data) // k
    remainder = len(data) % k
    
    lst = []
    loc1 = 0
    for i in range(k):
        loc2 = loc1 + base_size
        if i < remainder:
            loc2 += 1
        lst.append(data.iloc[loc1:loc2])
        loc1 = loc2
        
    return lst


In [9]:
list(range(5))

[0, 1, 2, 3, 4]

### Apply it to the Ames Housing data

In [11]:
# Make sure to concatenate the data again
ames_data = pd.concat([y, X], axis = 1)

In [26]:
# Apply kfolds() to ames_data with 5 folds

lst = kfolds(ames_data, 7)

length = 0
for l in lst:
    print(l['SalePrice_log'])
    length += len(l)

length == len(ames_data)

0      0.559876
1      0.212692
2      0.733795
3     -0.437232
4      1.014303
         ...   
204   -1.040965
205    0.198860
206   -0.368447
207   -0.419414
208    1.271046
Name: SalePrice_log, Length: 209, dtype: float64
209   -0.349383
210   -1.330143
211    0.274003
212    1.040977
213   -0.166327
         ...   
413   -0.929683
414    0.783699
415    0.207638
416   -0.272872
417    0.901655
Name: SalePrice_log, Length: 209, dtype: float64
418   -0.700995
419   -0.401722
420    0.533320
421    0.636729
422   -0.973604
         ...   
622   -0.528276
623    0.026637
624   -0.023636
625   -0.102946
626   -0.439021
Name: SalePrice_log, Length: 209, dtype: float64
627   -0.214939
628   -0.528276
629    0.026637
630   -0.741051
631    0.571854
         ...   
831   -0.247879
832    0.880618
833    0.004251
834   -0.438126
835   -0.661570
Name: SalePrice_log, Length: 209, dtype: float64
836    -0.206771
837    -1.279567
838    -0.366708
839    -0.613146
840    -0.437232
          ...  

True

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [36]:
test_errs = []
train_errs = []
k=5

lst = kfolds(ames_data, 7)

for n in range(k):
    # Split in train and test for the fold
    train = ames_data.drop(lst[n].index)
    test = lst[n]
    
    #print (len(ames_data), len(train), len(test))
    
    # Fit a linear regression model
    y_train = train['SalePrice_log']
    X_train = train.drop(['SalePrice_log'], axis=1)
    y_test = test['SalePrice_log']
    X_test = test.drop(['SalePrice_log'], axis=1)
    
    linreg = LinearRegression()
    model = linreg.fit(X_train, y_train)
    
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    
    train_errs.append(mean_squared_error(y_train, y_hat_train))
    test_errs.append(mean_squared_error(y_test, y_hat_test))
    
    
    
print(train_errs
print(test_errs)

[0.16790227661012638, 0.16436281740023548, 0.1542505146905541, 0.15682255705844436, 0.163178630547286]
[0.1240533152988408, 0.14699909246237688, 0.20979746326137466, 0.19554998165460483, 0.15757380134170557]


In [43]:
def kfolds_s(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
    return folds 

In [44]:
ames_folds = kfolds_s(ames_data, 5)

test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(ames_folds) if i!=n])
    test = ames_folds[n]
    # Fit a linear regression model
    linreg.fit(X_train, y_train)
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_residuals = y_hat_train - y_train
    test_residuals = y_hat_test - y_test
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

[0.163178630547286, 0.163178630547286, 0.163178630547286, 0.163178630547286, 0.163178630547286]
[0.15757380134170557, 0.15757380134170557, 0.15757380134170557, 0.15757380134170557, 0.15757380134170557]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [1]:
# Your code here
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

mse = make_scorer(mean_squared_error)
cvs = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')
np.sqrt(cvs)

NameError: name 'mean_squared_error' is not defined

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [59]:
# Your code here
print('sklearn cvs score mean:', 0-cvs.mean())

sklearn cvs score mean: 0.17702834210001125


##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 